In [19]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
from pathlib import Path
from pprint import pprint
# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# mycode
from repo import REPO
from ml_utilities.output_loader.job_output import SweepResult, JobResult
from ml_utilities.output_loader import create_job_output_loader
from ml_utilities.output_loader.plot import plot_sweep_summary, plot_data_log_values
from erank.mode_connectivity.instability_analysis import InstabilityAnalyzer

[autoreload of ml_utilities.logger failed: Traceback (most recent call last):
  File "/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/system/apps/userenv/beck/subspaces/lib/python3.10/site-packages/IPython/extensions/autorel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 13.1 CIFAR 10 ResNet Pretraining

With this notebook I start the pretraining runs on CIFAR10, which are then used for later finetuning for the stability analysis. 
I also start the finetuning runs here

In [20]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: dragonfly
  gpu_ids: [0,1]
  runs_per_gpu: 2
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [11,17,31]

sweep:
  type: grid
  axes: 
  - parameter: data.dataset_transforms.enable_transforms
    vals: [True, False]
  # - parameter: trainer.optimizer_scheduler.optimizer_kwargs.lr
  #   vals: [0.1, 0.075, 0.05, 0.025]
  # - parameter: trainer.optimizer_scheduler.optimizer_kwargs.weight_decay
  #   vals: [0.0, 0.001, 0.005]
  # - parameter: trainer.batch_size
  #   vals: [64, 256, 512]
    
start_num: 1

config:
  experiment_data:
    entity: jkuiml-fsl
    project_name: tflearning
    experiment_tag: '13.1'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet-B-pretrain
    experiment_dir: null
    experiment_notes: 
    job_name: null
    seed: 0
    hostname: null
    gpu_id: 0
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      notes: ${config.experiment_data.experiment_notes}
      group: ${config.experiment_data.experiment_tag}
      job_type: ${config.experiment_data.experiment_type}
    watch:
      log: null
      log_freq:

  model:
    model_cfg: resnet20-cifar10-B
    # name: resnet
    # model_kwargs:
    #   in_channels: 3
    #   act_fn: relu
    #   residual_option: B
    #   input_layer_config:
    #     kernel_size: 3
    #     out_channels: 16
    #     bias: false
    #     batch_norm: true
    #     stride: 1
    #     padding: 1
    #   resnet_blocks_config:
    #     - out_channels: 16
    #       num_residual_blocks: 3
    #     - out_channels: 32
    #       num_residual_blocks: 3
    #     - out_channels: 64
    #       num_residual_blocks: 3
    #   linear_output_units:
    #     - 10
    init_model: null

  trainer:
    training_setup: supervised
    n_steps: 64e3
    log_train_step_every: 1
    log_additional_train_step_every_multiplier: 1
    log_additional_logs: true
    val_every: 500
    save_every: 1000
    early_stopping_patience: 64e3
    batch_size: 128
    optimizer_scheduler:
      optimizer_name: SGD
      optimizer_kwargs:
        lr: 0.01 #0.1
        momentum: 0.9
        weight_decay: 0.0001
      lr_scheduler_name: MultiStepLR
      lr_scheduler_kwargs:
        milestones: [32e3, 48e3]
        gamma: 0.1

    loss: crossentropy
    metrics:
    - Accuracy
    num_workers: 4
    verbose: false
  data:
    dataset: cifar10
    dataset_kwargs:
      data_root_path: /system/user/beck/pwbeck/data
    dataset_split:
      train_val_split: 0.9
      # restrict_n_samples_train_task: 100
    dataset_transforms:
      image_transforms:
      - RandomHorizontalFlip
      - RandomCrop:
          size: 32
          padding: 4
      tensor_transforms: 
      joint_tensor_transforms: 
      enable_transforms: True
"""
cfg = OmegaConf.create(config_yaml)

In [21]:
# run_command = REPO.create_experiment(cfg, override=False)
# print(run_command)

### Sweep result

In [22]:
sweepr = REPO.get_output_loader(cfg)
print(sweepr)

Exp. Tag(start_num): 13.1(1)
Exp. Name: cifar10-13.1.1-resnet-B-pretrain
Training setup: supervised
Model name: resnet20-cifar10-B
Dataset name: cifar10
Sweep type: grid
  data.dataset_transforms.enable_transforms: [True, False]
Seeds: [11, 17, 31]
Num. jobs: 6
Config updated: 2023-01-12 16:36:01
Sweep started:  2023-01-12 16:39:14



In [23]:
failed_jobs = sweepr.get_failed_jobs()
failed_jobs # No failed jobs

([], {})

In [24]:
pprint(sweepr.available_log_columns)

{'_common_cols': ['epoch', 'train_step', 'log_step']}


In [25]:
sw_summary = sweepr.get_summary()
sw_summary

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

# 13.5 CIFAR10 Resnet enable_transforms=True finetuning

In [ ]:
# get a single pretraining job
df, jobs = sweepr.query_jobs({'seed':31, 'data.dataset_transforms.enable_transforms': True})
df

,best_step,best_val_score,data.dataset_transforms.enable_transforms,seed
cifar10-13.1.1-resnet-B-pretrain--enable_transforms-1-seed-31--230112_163938,62000,0.8822,True,31


In [ ]:
pretrain_job_et = jobs[0]
pretrain_job_et

JobResult(cifar10-13.1.1-resnet-B-pretrain--enable_transforms-1-seed-31--230112_163938)

In [ ]:
print(pretrain_job_et.directory)

/system/user/publicwork/beck/projects/regularization/erank/outputs/cifar10-13.1.1-resnet-B-pretrain--230112_163914/outputs/cifar10-13.1.1-resnet-B-pretrain--enable_transforms-1-seed-31--230112_163938


In [ ]:
# determine pretrain indices, remaining training time and lr schedule milestones
pretrain_idxes = np.array(pretrain_job_et.available_model_checkpoint_indices[::6])
pretrain_idxes.tolist()

[0, 6000, 12000, 18000, 24000, 30000, 36000, 42000, 48000, 54000, 60000]

In [ ]:
total_steps = cfg.config.trainer.n_steps
remaining_training_steps = total_steps - np.array(pretrain_idxes)
remaining_training_steps.astype(int).tolist()

[64000, 58000, 52000, 46000, 40000, 34000, 28000, 22000, 16000, 10000, 4000]

In [ ]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: dragonfly
  gpu_ids: [0,1]
  runs_per_gpu: 2
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [21,22]

sweep:
  type: line
  axes: 
  - parameter: trainer.n_steps # remaining steps to train
    vals: [64000, 58000, 52000, 46000, 40000, 34000, 28000, 22000, 16000, 10000, 4000]
  - parameter: model.pretrain_idx
    vals: [0, 6000, 12000, 18000, 24000, 30000, 36000, 42000, 48000, 54000, 60000]

    
start_num: 0

config:
  experiment_data:
    entity: jkuiml-fsl
    project_name: tflearning
    experiment_tag: '13.5'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet-B-finetune-enableT
    experiment_dir: null
    experiment_notes: 
    job_name: null
    seed: 0
    hostname: null
    gpu_id: 0
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      notes: ${config.experiment_data.experiment_notes}
      group: ${config.experiment_data.experiment_tag}
      job_type: ${config.experiment_data.experiment_type}
    watch:
      log: null
      log_freq:

  model:
    model_cfg: resnet20-cifar10-B
    # name: resnet
    # model_kwargs:
    #   in_channels: 3
    #   act_fn: relu
    #   residual_option: B
    #   input_layer_config:
    #     kernel_size: 3
    #     out_channels: 16
    #     bias: false
    #     batch_norm: true
    #     stride: 1
    #     padding: 1
    #   resnet_blocks_config:
    #     - out_channels: 16
    #       num_residual_blocks: 3
    #     - out_channels: 32
    #       num_residual_blocks: 3
    #     - out_channels: 64
    #       num_residual_blocks: 3
    #   linear_output_units:
    #     - 10
    pretrain_idx: 0
    init_model: /system/user/publicwork/beck/projects/regularization/erank/outputs/cifar10-13.1.1-resnet-B-pretrain--230112_163914/outputs/cifar10-13.1.1-resnet-B-pretrain--enable_transforms-1-seed-31--230112_163938/model_step_${config.model.pretrain_idx}.p


  trainer:
    training_setup: supervised
    n_steps: 64e3
    log_train_step_every: 1
    log_additional_train_step_every_multiplier: 1
    log_additional_logs: true
    val_every: 500
    save_every: 1000
    early_stopping_patience: 64e3
    batch_size: 128
    optimizer_scheduler:
      optimizer_name: SGD
      optimizer_kwargs:
        lr: 0.01 #0.1
        momentum: 0.9
        weight_decay: 0.0001
      lr_scheduler_name: MultiStepLR
      lr_scheduler_kwargs:
        milestones: [32e3, 48e3]
        gamma: 0.1

    loss: crossentropy
    metrics:
    - Accuracy
    num_workers: 4
    verbose: false
  data:
    dataset: cifar10
    dataset_kwargs:
      data_root_path: /system/user/beck/pwbeck/data
    dataset_split:
      train_val_split: 0.9
      # restrict_n_samples_train_task: 100
    dataset_transforms:
      image_transforms:
      - RandomHorizontalFlip
      - RandomCrop:
          size: 32
          padding: 4
      tensor_transforms: 
      joint_tensor_transforms: 
      enable_transforms: True
"""
cfg = OmegaConf.create(config_yaml)

In [ ]:
# run_command = REPO.create_experiment(cfg, override=False)
# print(run_command)

In [ ]:
finetunesw5 = REPO.get_output_loader(cfg)
print(finetunesw5)

Exp. Tag(start_num): 13.5(0)
Exp. Name: cifar10-13.5.0-resnet-B-finetune-enableT
Training setup: supervised
Model name: resnet20-cifar10-B
Dataset name: cifar10
Sweep type: line
  trainer.n_steps: [64000, 58000, 52000, 46000, 40000, 34000, 28000, 22000, 16000, 10000, 4000]
  model.pretrain_idx: [0, 6000, 12000, 18000, 24000, 30000, 36000, 42000, 48000, 54000, 60000]
Seeds: [21, 22]
Num. jobs: 22
Config updated: 2023-01-13 10:33:01
Sweep started:  2023-01-13 10:33:37



In [ ]:
fdir = finetunesw5.directory
print(fdir)

/system/user/publicwork/beck/projects/regularization/erank/outputs/cifar10-13.5.0-resnet-B-finetune-enableT--230113_103337


In [ ]:
insta5 = InstabilityAnalyzer.reload(finetunesw5.directory, instability_folder_suffix='1')
insta5.combined_results_dfs['datasets']

datasets                                                                                                                    val  \
score                                                                                                      interpolation_scores   
alpha                                                                                                                      0.00   
default_params init_model_idx_k job                                                seeds    model_idxes                           
default_params 0                cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (54000, 35500)             0.861328   
                                                                                            (64000, 64000)             0.861914   
               6000             cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (34000, 48500)             0.873828   
                                                                                            (58000, 58000)             0.873047   
               12000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (46000, 33500)             0.879102   
                                                                                            (52000, 52000)             0.878711   
               18000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (33500, 40500)             0.875000   
                                                                                            (46000, 46000)             0.871484   
               24000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (34500, 40000)             0.880469   
                                                                                            (40000, 40000)             0.880664   
               30000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (34000, 33500)             0.875586   
                                                                                            (34000, 34000)             0.875195   
               36000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (24500, 23000)             0.870898   
                                                                                            (28000, 28000)             0.870313   
               42000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (9000, 19500)              0.867969   
                                                                                            (22000, 22000)             0.868750   
               48000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (0, 0)                     0.872070   
                                                                                            (16000, 16000)             0.873047   
               54000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (0, 0)                     0.871094   
                                                                                            (10000, 10000)             0.870703   
               60000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (0, 0)                     0.871680   
                                                                                            (4000, 4000)               0.871094   

datasets                                                                                                              \
score                                                                                                                  
alpha                                                                                                           0.25   
default_params init_model_idx_k job                                                seeds    model_idxes                
default_params 0                cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (54000, 35500)  0.708

In [ ]:
insta5 = InstabilityAnalyzer.reload(finetunesw5.directory, instability_folder_suffix='2')
insta5.combined_results_dfs['datasets']

datasets                                                                                                                    val  \
score                                                                                                      interpolation_scores   
alpha                                                                                                                      0.00   
default_params init_model_idx_k job                                                seeds    model_idxes                           
default_params 0                cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (54000, 35500)             0.862109   
                                                                                            (64000, 64000)             0.861328   
               6000             cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (34000, 48500)             0.873633   
                                                                                            (58000, 58000)             0.873633   
               12000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (46000, 33500)             0.882422   
                                                                                            (52000, 52000)             0.881836   
               18000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (33500, 40500)             0.872070   
                                                                                            (46000, 46000)             0.871680   
               24000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (34500, 40000)             0.880273   
                                                                                            (40000, 40000)             0.879883   
               30000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (34000, 33500)             0.874805   
                                                                                            (34000, 34000)             0.875391   
               36000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (24500, 23000)             0.871094   
                                                                                            (28000, 28000)             0.871289   
               42000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (9000, 19500)              0.868945   
                                                                                            (22000, 22000)             0.868164   
               48000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (0, 0)                     0.871289   
                                                                                            (16000, 16000)             0.871484   
               54000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (0, 0)                     0.870703   
                                                                                            (10000, 10000)             0.870898   
               60000            cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (0, 0)                     0.870898   
                                                                                            (4000, 4000)               0.871680   

datasets                                                                                                              \
score                                                                                                                  
alpha                                                                                                           0.25   
default_params init_model_idx_k job                                                seeds    model_idxes                
default_params 0                cifar10-13.5.0-resnet-B-finetune-enableT--n_ste... (21, 22) (54000, 35500)  0.708

In [ ]:
# finetunesw5.get_summary()

In [ ]:
idx0job = finetunesw5.get_jobs('_idx-0-seed-21')[0]
idx0job.get_data_log('val')

,log_step,epoch,train_step,loss_CrossEntropyLoss,Accuracy,time_last_val_epoch_in_s
0,0,0,0,2.369769,0.0902,0.000000
1,501,2,500,1.479348,0.4630,2.620911
2,1003,3,1000,1.488813,0.4312,0.439410
3,1506,5,1500,1.420181,0.4884,0.526924
4,2008,6,2000,1.409338,0.4902,0.519385
...,...,...,...,...,...,...
124,62299,177,62000,0.435284,0.8678,0.553723
125,62801,178,62500,0.438549,0.8672,0.527406
126,63303,179,63000,0.442000,0.8658,0.505147
127,63806,181,63500,0.464248,0.8670,0.532207


In [ ]:
idx0job = finetunesw5.get_jobs('_idx-0-seed-22')[0]
idx0job.get_data_log('val')

,log_step,epoch,train_step,loss_CrossEntropyLoss,Accuracy,time_last_val_epoch_in_s
0,0,0,0,2.366740,0.0902,0.000000
1,501,2,500,1.338886,0.5220,2.748828
2,1003,3,1000,1.391599,0.5030,0.542789
3,1506,5,1500,1.156231,0.5908,0.534415
4,2008,6,2000,1.055278,0.6218,0.566908
...,...,...,...,...,...,...
124,62299,177,62000,0.437943,0.8754,0.512799
125,62801,178,62500,0.465370,0.8764,0.532349
126,63303,179,63000,0.437721,0.8748,0.541527
127,63806,181,63500,0.442293,0.8764,0.549921


# 13.6 CIFAR10 Resnet enable_transforms=False finetuning

In [29]:
# get a single pretraining job
df, jobs = sweepr.query_jobs({'seed':31, 'data.dataset_transforms.enable_transforms': False})
df

,best_step,best_val_score,data.dataset_transforms.enable_transforms,seed
cifar10-13.1.1-resnet-B-pretrain--enable_transforms-0-seed-31--230112_163941,25500,0.798,False,31


In [30]:
pretrain_job_dt = jobs[0]
print(pretrain_job_dt.directory)

/system/user/publicwork/beck/projects/regularization/erank/outputs/cifar10-13.1.1-resnet-B-pretrain--230112_163914/outputs/cifar10-13.1.1-resnet-B-pretrain--enable_transforms-0-seed-31--230112_163941


In [49]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: dragonfly
  gpu_ids: [2,3]
  runs_per_gpu: 2
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [21,22]

sweep:
  type: line
  axes: 
  - parameter: trainer.n_steps # remaining steps to train
    vals: [64000, 58000, 52000, 46000, 40000, 34000, 28000, 22000, 16000, 10000, 4000]
  - parameter: model.pretrain_idx
    vals: [0, 6000, 12000, 18000, 24000, 30000, 36000, 42000, 48000, 54000, 60000]

    
start_num: 0

config:
  experiment_data:
    entity: jkuiml-fsl
    project_name: tflearning
    experiment_tag: '13.6'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet-B-finetune-disableT
    experiment_dir: null
    experiment_notes: 
    job_name: null
    seed: 0
    hostname: null
    gpu_id: 0
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      notes: ${config.experiment_data.experiment_notes}
      group: ${config.experiment_data.experiment_tag}
      job_type: ${config.experiment_data.experiment_type}
    watch:
      log: null
      log_freq:

  model:
    model_cfg: resnet20-cifar10-B
    # name: resnet
    # model_kwargs:
    #   in_channels: 3
    #   act_fn: relu
    #   residual_option: B
    #   input_layer_config:
    #     kernel_size: 3
    #     out_channels: 16
    #     bias: false
    #     batch_norm: true
    #     stride: 1
    #     padding: 1
    #   resnet_blocks_config:
    #     - out_channels: 16
    #       num_residual_blocks: 3
    #     - out_channels: 32
    #       num_residual_blocks: 3
    #     - out_channels: 64
    #       num_residual_blocks: 3
    #   linear_output_units:
    #     - 10
    pretrain_idx: 0
    init_model: /system/user/publicwork/beck/projects/regularization/erank/outputs/cifar10-13.1.1-resnet-B-pretrain--230112_163914/outputs/cifar10-13.1.1-resnet-B-pretrain--enable_transforms-0-seed-31--230112_163941/model_step_${config.model.pretrain_idx}.p


  trainer:
    training_setup: supervised
    n_steps: 64e3
    log_train_step_every: 1
    log_additional_train_step_every_multiplier: 1
    log_additional_logs: true
    val_every: 500
    save_every: 1000
    early_stopping_patience: 64e3
    batch_size: 128
    optimizer_scheduler:
      optimizer_name: SGD
      optimizer_kwargs:
        lr: 0.01 #0.1
        momentum: 0.9
        weight_decay: 0.0001
      lr_scheduler_name: MultiStepLR
      lr_scheduler_kwargs:
        milestones: [32e3, 48e3]
        gamma: 0.1

    loss: crossentropy
    metrics:
    - Accuracy
    num_workers: 4
    verbose: false
  data:
    dataset: cifar10
    dataset_kwargs:
      data_root_path: /system/user/beck/pwbeck/data
    dataset_split:
      train_val_split: 0.9
      # restrict_n_samples_train_task: 100
    dataset_transforms:
      image_transforms:
      - RandomHorizontalFlip
      - RandomCrop:
          size: 32
          padding: 4
      tensor_transforms: 
      joint_tensor_transforms: 
      enable_transforms: True
"""
cfg = OmegaConf.create(config_yaml)

In [50]:
# run_command = REPO.create_experiment(cfg, override=False)
# print(run_command)

In [54]:
finetunesw6 = REPO.get_output_loader(cfg)

In [55]:
print(finetunesw6.directory)

/system/user/publicwork/beck/projects/regularization/erank/outputs/cifar10-13.6.0-resnet-B-finetune-disableT--230113_103818


In [57]:
insta6 = InstabilityAnalyzer.reload(finetunesw6.directory, instability_folder_suffix='0')

In [61]:
insta6.combined_results_dfs['datasets']

datasets                                                                                                                    val  \
score                                                                                                      interpolation_scores   
alpha                                                                                                                      0.00   
default_params init_model_idx_k job                                                seeds    model_idxes                           
default_params 0                cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (40000, 38500)               0.0984   
                                                                                            (64000, 64000)               0.0984   
               6000             cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (56500, 33000)               0.1152   
                                                                                            (58000, 58000)               0.1152   
               12000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (35000, 40500)               0.1244   
                                                                                            (52000, 52000)               0.1246   
               18000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (34500, 44500)               0.1804   
                                                                                            (46000, 46000)               0.1802   
               24000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (32500, 36500)               0.2032   
                                                                                            (40000, 40000)               0.2030   
               30000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (34000, 34000)               0.2228   
                                                                                            (34000, 34000)               0.2228   
               36000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (28000, 28000)               0.1964   
                                                                                            (28000, 28000)               0.1970   
               42000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (15000, 21500)               0.2622   
                                                                                            (22000, 22000)               0.2620   
               48000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (15000, 16000)               0.1824   
                                                                                            (16000, 16000)               0.1816   
               54000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (10000, 10000)               0.2618   
                                                                                            (10000, 10000)               0.2626   
               60000            cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (3500, 3500)                 0.2378   
                                                                                            (4000, 4000)                 0.2376   

datasets                                                                                                            \
score                                                                                                                
alpha                                                                                                         0.25   
default_params init_model_idx_k job                                                seeds    model_idxes              
default_params 0                cifar10-13.6.0-resnet-B-finetune-disableT--n_st... (21, 22) (40000, 38500)  0.1308   
   